## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

TensorFlow version: 2.7.0
1 Physical GPUs, 1 Logical GPUs


In [2]:
import pathlib
import sys


sys.path.insert(0,'..')
from src.load_dataset_fft_aug import get_splitter_dataloaders_fft

# Build model

In [3]:
layers.Conv2D(filters=32,
            kernel_size=3,
            strides=1,
            padding='same',
            activation='relu')(tf.random.normal([1, 28, 28, 1])).shape

TensorShape([1, 28, 28, 32])

In [4]:
layers.LSTM(units=40, return_sequences=True)(tf.random.normal([1, 28, 280])).shape

TensorShape([1, 28, 40])

In [5]:
model = tf.keras.models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation="relu", input_shape=(201, 498)),
    layers.MaxPool1D(pool_size=4, strides=4),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.MaxPool1D(pool_size=2, strides=2),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.Dropout(0.5),
    
    layers.LSTM(units=40, return_sequences=True),
    
    layers.MaxPool1D(pool_size=2, strides=2),
    layers.Flatten(),
    
    layers.Dense(2)
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# L1 loss
model.compile(optimizer=optimizer,
            loss="mae",
            metrics=["mae"]
            )

In [6]:
import numpy as np
tf.constant([0.0, 0.0])
x = tf.random.normal([1, 201, 498])

In [7]:
model.fit(x=x, y=tf.constant([[0.0, 0.0]]), epochs=1)

1/1 [==============================] - 3s 3s/step - loss: 0.2470 - mae: 0.2470


### Model Training

Bring dataset

In [8]:
_, _, train, val = get_splitter_dataloaders_fft(noise_attenuation=0.00001)

Caching dataset: 100%|██████████| 1144/1144 [00:16<00:00, 68.05it/s]


Build a generator for the dataset in order to feed it to the model directly.

In [9]:
# y_train = []
# for i in range(len(train)):
#     y_train.append(train[i][1])

# y_val = []
# for i in range(len(val)):
#     y_val.append(val[i][1])

In [10]:
def train_generator():
    for i in range(len(train)):
        instance = train[i]
        yield instance[0], instance[1]
        
def val_generator():
    for i in range(len(val)):
        instance = val[i]
        yield instance[0], instance[1]
        
BATCH_SIZE = 16
train_dataset = tf.data.Dataset.from_generator(train_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_generator(val_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)

Fit the model 😃

In [11]:
model.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
286/286 [==============================] - 18s 59ms/step - loss: 1.4965 - mae: 1.4965 - val_loss: 1.1138 - val_mae: 1.1138
Epoch 2/100
286/286 [==============================] - 15s 52ms/step - loss: 1.3093 - mae: 1.3093 - val_loss: 0.9861 - val_mae: 0.9861
Epoch 3/100
286/286 [==============================] - 14s 49ms/step - loss: 1.2484 - mae: 1.2484 - val_loss: 0.8515 - val_mae: 0.8515
Epoch 4/100
286/286 [==============================] - 15s 51ms/step - loss: 1.1988 - mae: 1.1988 - val_loss: 0.8619 - val_mae: 0.8619
Epoch 5/100
286/286 [==============================] - 14s 49ms/step - loss: 1.1725 - mae: 1.1725 - val_loss: 0.9692 - val_mae: 0.9692
Epoch 6/100
286/286 [==============================] - 14s 50ms/step - loss: 1.1798 - mae: 1.1798 - val_loss: 0.8629 - val_mae: 0.8629
Epoch 7/100
286/286 [==============================] - 20s 71ms/step - loss: 1.1610 - mae: 1.1610 - val_loss: 0.9258 - val_mae: 0.9258
Epoch 8/100
286/286 [==============================] - 

In [ ]:
model.fit(val_dataset, epochs=2)

### Save model

In [ ]:
save_path = "saved/tf_model"
# tf.saved_model.save(model, save_path)

run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
FREQS = 201
TIME_STEPS = 498
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, FREQS, TIME_STEPS], model.inputs[0].dtype))
model.save(save_path, save_format="tf", signatures=concrete_func)

INFO:tensorflow:Assets written to: model_not_trained/assets


INFO:tensorflow:Assets written to: model_not_trained/assets


# Tinify model

In [34]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_path)

save_path_tflite = os.path.basename(save_path) + ".tflite"

converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_model = converter.convert()
tflite_model_file = pathlib.Path(save_path_tflite)
tflite_model_file.write_bytes(tflite_model)

Estimated count of arithmetic ops: 20.144 M  ops, equivalently 10.072 M  MACs


97056